# Google Art & Culture - Case study using CRISPS-DM

#### Autors: Manuel Alejandro Aponte, Cristian Beltran, Maria Paula Peña

In this notebook it will webscraping of the page Google Art & Culture

## Objectives
The objective of this notebooks is:

* Download images using webscraping.
* Download images metadata.
* Store all information in a datasheet.

## Prerequisites

* Familiarity with python 
* Lastest version of Google WebDriver, Source: https://chromedriver.chromium.org/
* Install python packages.
* Use VPN (Recomended)

## Background 
This notebook belongs to Google Art & Culture Case Study using CRIPS-DM, where would be include process such as webscraping, exploratory data analysis, ML classificators and dashboards. 

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#Import project packages
from src.Scraper.Scraper import Scraper
from src.Parallel.Parallel import parallel
from src.paths.paths import PATHS
#Import packages
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import json

# Global Settings

In [5]:
#Colors of sections to be webscraped
COLORS = ["WHITE","PINK","YELLOW","PURPLE","BLUE","TEAL","GREEN","ORANGE","RED","BROWN","BLACK"] 

#Webdriver path
DRIVER_PATH = r"../chromedriver.exe"

#Folder of data resources
DATA_RAW_FOLDER = PATHS['data_raw']
DATA_PROCESSED_FOLDER = PATHS['data_processed']
DATA_FINAL_FOLDER = PATHS['data_final']


# Webscraping Attributes Extraction (Phase 1)

In [25]:
def webscraping(color:str):
    """Perform webscraping of Google Art & Culture web page

    Parameters
    ----------
    color : str
        the color of target page
    """
    scraper = Scraper(DRIVER_PATH,color)
    scraper.open()
    data = scraper.scraping_data()
    return data
    
    
webscraping_data = parallel(webscraping,COLORS)

Running


inspect some data

In [26]:
#Print WHITE collection
print(webscraping_data[0][0:2])
#Print PINK collection
print(webscraping_data[1][0:2])

[{'url': 'https://lh3.googleusercontent.com/ci/AC_FhM8vjjpeTbkrTcxxGq8UFhR9du2a5U9RyRnFs2tI13Cy6WehwovfoyrAfWvLbpwI52I69j3TG4A=w313-c-h313-fcrop64=1,00000d37ffff8ce4-rw-v1', 'data': '{"Title":" Symphony in White, No. 1 The White Girl","Creator":" James McNeill Whistler","Date Created":" 1862","External Link":"  For more information about this and thousands of other works of art in the NGA collection, please visit\xa0http//www.nga.gov/","Medium":" oil on canvas","Object Credit":" Harris Whittemore Collection","Dimensions":" overall 213 x 107.9 cm (83 7/8 x 42 1/2 in.)\\u000b\\u000bframed 244.2 x 136.5 x 8.3 cm (96 1/8 x 53 3/4 x 3 1/4 in.)","Classification":" Painting","Artist School":" American","Artist Nationality":" American","Artist Details":" American, 1834 - 1903","museum":"National Gallery of Art, Washington DC","location":"Washington, DC, United States"}', 'category': 'WHITE'}, {'url': 'https://lh3.googleusercontent.com/ci/AC_FhM9fS_Widg084SOpVx3jOe9fAldy6MDwA3K4mhLVE-0fLp0cY8J3

Merge Collections into one

In [27]:
flatten_data = [item for sublist in webscraping_data for item in sublist]

In [28]:
flatten_data[0:2]

[{'url': 'https://lh3.googleusercontent.com/ci/AC_FhM8vjjpeTbkrTcxxGq8UFhR9du2a5U9RyRnFs2tI13Cy6WehwovfoyrAfWvLbpwI52I69j3TG4A=w313-c-h313-fcrop64=1,00000d37ffff8ce4-rw-v1',
  'data': '{"Title":" Symphony in White, No. 1 The White Girl","Creator":" James McNeill Whistler","Date Created":" 1862","External Link":"  For more information about this and thousands of other works of art in the NGA collection, please visit\xa0http//www.nga.gov/","Medium":" oil on canvas","Object Credit":" Harris Whittemore Collection","Dimensions":" overall 213 x 107.9 cm (83 7/8 x 42 1/2 in.)\\u000b\\u000bframed 244.2 x 136.5 x 8.3 cm (96 1/8 x 53 3/4 x 3 1/4 in.)","Classification":" Painting","Artist School":" American","Artist Nationality":" American","Artist Details":" American, 1834 - 1903","museum":"National Gallery of Art, Washington DC","location":"Washington, DC, United States"}',
  'category': 'WHITE'},
 {'url': 'https://lh3.googleusercontent.com/ci/AC_FhM9fS_Widg084SOpVx3jOe9fAldy6MDwA3K4mhLVE-0fLp0

In [29]:
df = pd.DataFrame(flatten_data)
df['index'] = df.index
df.to_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv', index = False)  

In [30]:
num = len(df)
print(f'There are {num} extracted pictures')

There are 11077 extracted pictures


# Webscraping Image Extraction (Phase 2)

Get the image_url and the identify key (index)

In [6]:
   
df = pd.read_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv')
df = df[['index','url']]   
data = list(df.itertuples(index=False, name=None)) # Convert data into tuples

Apply parallel transformation for download files 

In [7]:
def get_image(data):
    id = data[0]
    link = data[1]
    path = f'{DATA_FINAL_FOLDER}/img/{id}.jpg'
    img = requests.get(link).content
    with open(path, 'wb') as handler:
        handler.write(img)
    return (id,f'{id}.jpg')

image_files = parallel(get_image,data) 
print(image_files[0:3])

Running
[(0, '0.jpg'), (1, '1.jpg'), (2, '2.jpg')]


Convert data into df and export it

In [8]:
df_images = pd.DataFrame(image_files, columns=['index','filename']) #Parse data into df
df_images.to_csv(f'{DATA_PROCESSED_FOLDER}/picture_files.csv', index = False)  # Export

# Transform

* Filtering, cleansing, de-duplicating, validating, and authenticating the data.
* Performing calculations, translations, or summarizations based on the raw data. This can  include changing row and column headers for consistency, converting currencies or other units of measurement, editing text strings, and more.
* Ensure data quality
* Formatting the data into tables or joined tables to match the schema of the target data warehouse.

### Check Data integrity

In [9]:
df_original = pd.read_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv')
df_images = pd.read_csv(f'{DATA_PROCESSED_FOLDER}/picture_files.csv')
df = df_original.merge(df_images, left_on='index', right_on='index')
print(df.head())

                                                 url  \
0  https://lh3.googleusercontent.com/ci/AC_FhM8vj...   
1  https://lh3.googleusercontent.com/ci/AC_FhM9fS...   
2  https://lh3.googleusercontent.com/ci/AC_FhM8fh...   
3  https://lh3.googleusercontent.com/ci/AC_FhM9VK...   
4  https://lh3.googleusercontent.com/ci/AC_FhM9KL...   

                                                data category  index filename  
0  {"Title":" Symphony in White, No. 1 The White ...    WHITE      0    0.jpg  
1  {"Title":" Summer evening on Skagen Sønderstra...    WHITE      1    1.jpg  
2  {"Title":" Composition with red, yellow and bl...    WHITE      2    2.jpg  
3  {"Title":" The Magpie","Date Created":" 1868 -...    WHITE      3    3.jpg  
4  {"Title":" The Cradle","Date Created":" 1872",...    WHITE      4    4.jpg  


In [10]:
print('Final Data Shape:',df.shape)
print('Original Data Shape:',df_original.shape)
print('Image Files Data Shape:',df_images.shape)

Final Data Shape: (11077, 5)
Original Data Shape: (11077, 4)
Image Files Data Shape: (11077, 2)


In [ ]:
df.info()

In [11]:
print('Exist Duplicated:', df['index'].duplicated().any())
print('Exist Duplicated:', df['url'].duplicated().any())

Exist Duplicated: False
Exist Duplicated: False


In [12]:
df.isnull().sum()

url         0
data        0
category    0
index       0
filename    0
dtype: int64

### Parse data column into a df

In [13]:

data_string = list(df.data.values)
data_json = list(map(lambda string_json:json.loads(string_json),data_string))

In [15]:
data_df = json_normalize(data_json)
data_df.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_16880\3295387368.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  data_df = json_normalize(data_json)


,Title,Creator,Date Created,External Link,Medium,Object Credit,Dimensions,Classification,Artist School,Artist Nationality,...,National Museum of Women in the Arts’ Exhibition,Permanent exhibition,Additional Artist Details,Artist Death Place,Commission,Additional Artist Name,Provenance Link,Frame,Original Title (portuguese),Material(s) / Technique(s)
0,"Symphony in White, No. 1 The White Girl",James McNeill Whistler,1862,For more information about this and thousand...,oil on canvas,Harris Whittemore Collection,overall 213 x 107.9 cm (83 7/8 x 42 1/2 in.) ...,Painting,American,American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summer evening on Skagen Sønderstrand,Peder Severin Krøyer,1893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Composition with red, yellow and blue",Piet Mondriaan,1927,Collection searcher,Oil on canvas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
full_df = pd.concat([df,data_df ],axis=1)
full_df = full_df.drop(['data'],axis=1)
full_df.head(3)

,url,category,index,filename,Title,Creator,Date Created,External Link,Medium,Object Credit,...,National Museum of Women in the Arts’ Exhibition,Permanent exhibition,Additional Artist Details,Artist Death Place,Commission,Additional Artist Name,Provenance Link,Frame,Original Title (portuguese),Material(s) / Technique(s)
0,https://lh3.googleusercontent.com/ci/AC_FhM8vj...,WHITE,0,0.jpg,"Symphony in White, No. 1 The White Girl",James McNeill Whistler,1862,For more information about this and thousand...,oil on canvas,Harris Whittemore Collection,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://lh3.googleusercontent.com/ci/AC_FhM9fS...,WHITE,1,1.jpg,Summer evening on Skagen Sønderstrand,Peder Severin Krøyer,1893,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://lh3.googleusercontent.com/ci/AC_FhM8fh...,WHITE,2,2.jpg,"Composition with red, yellow and blue",Piet Mondriaan,1927,Collection searcher,Oil on canvas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check again data integrity

In [17]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11077 entries, 0 to 11076
Columns: 518 entries, url to Material(s) / Technique(s)
dtypes: int64(1), object(517)
memory usage: 43.9+ MB


# Export all information 

In [18]:
full_df.to_csv(f'{DATA_PROCESSED_FOLDER}/picture_data.csv', index = False)